[![image](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/gdslab/d2spy/blob/main/docs/guides/notebooks/03_adding_multiple_data_products.ipynb)
[![Jupyter Notebook](https://img.shields.io/badge/Open%20in%20JuypterHub%20-%20%233776AB?logo=jupyter&logoColor=%23F37626&labelColor=%23F5F5F5)](https://lab.d2s.org/hub/user-redirect/lab/tree/tutorials/03_adding_multiple_data_products.ipynb)

# Adding multiple data products to an existing flight
*This guide will walk you through the steps for adding multiple data products, of different data types, to an existing flight.*

To get started, you will need to import the `Workspace` modules.

In [ ]:
# Uncomment and run the following line if working out of Google Colab
# !pip install d2spy

In [ ]:
from d2spy.workspace import Workspace

All of your D2S data can be accessed through a D2S "workspace." The [`Workspace`](https://py.d2s.org/workspace/) module's [`connect`](https://py.d2s.org/workspace/#d2spy.workspace.Workspace.connect) method can be used to login to a D2S instance and connect to your workspace in one go. Behind the scenes, the [`Auth`](https://py.d2s.org/auth/) module will be used to handle authenticating with D2S and requesting an authorization token. You will need to provide `connect` with the URL to your D2S instance and enter your password when prompted.

In [ ]:
# Example of connecting to a workspace for a local D2S instance
workspace = Workspace.connect("http://localhost:8000", "yourD2Semail@example.com")

The Workspace [`get_project`](https://py.d2s.org/workspace/#d2spy.workspace.Workspace.get_project) method will use a project ID to retrieve a specific project on your D2S instance. Once we have the project, we can use the project's [`get_flight`](https://py.d2s.org/project/#d2spy.models.project.Project.get_flight) method to find a specific flight by its ID.

You can find the project ID in the URL of your project page on your D2S instance, or by using the [`get_projects`](https://py.d2s.org/workspace/#d2spy.workspace.Workspace.get_projects) method to retrieve all projects and identifying the project ID from the returned list. Similarly, the flight ID can be found in the URL of your flight's data product page on your D2S instance, or by using the [`get_flights`](https://py.d2s.org/project/#d2spy.models.project.Project.get_flights) method. See the second guide, [Accessing your workspace projects, flights, and data products](https://py.d2s.org/guides/notebooks/02_accessing_your_workspace/), for detailed examples using these methods.

In [ ]:
# Find specific project in our workspace
project = workspace.get_project("a538eb21-cb9a-488b-989c-d82098cd843d")
if not project:
    print("Unable to find project")

# Find specific flight in our project
flight = project.get_flight("526e316c-3373-435c-a0f0-3781ced28508")
if not flight:
    print("Unable to find flight")

The Flight module's [`add_data_product`](https://py.d2s.org/flight/#d2spy.models.flight.Flight.add_data_product) method will be used to upload multiple data products located on the local machine.

**Important**

The `data_type` has some limits on what values will be accepted. `data_type` expects the values "dsm", "point_cloud", or "ortho". It can be provided with an alternative value if none of these options are suitable. 

Refer to the [documentation](https://py.d2s.org/flight/#d2spy.models.flight.Flight.add_data_product) for more details.

In [ ]:
# List of local data products that will be uploaded
data_products_to_upload = [
    {"filepath": "/full/path/to/my/ortho_data_product.tif", "data_type": "ortho"},
    {"filepath": "/full/path/to/my/dsm_data_product.tif", "data_type": "dsm"},
    {"filepath": "/full/path/to/my/point_cloud_data_product.tif", "data_type": "point_cloud"}
]

# Upload each data product to D2S instance
for data_product in data_products_to_upload:
    flight.add_data_product(
        filepath=data_product["filepath"],
        data_type=data_product["data_type"]
    )

When D2S has finished processing the upload the Flight module's [`get_data_products`](https://py.d2s.org/flight/#d2spy.models.flight.Flight.get_data_products) method can be used to find the uploaded data products' static URLs.

In [ ]:
data_products = flight.get_data_products()

# If an uploaded data product is missing from the output it may need more time to process on D2S
for data_product in data_products:
    print(data_product.url)

These are all the methods you need to know for viewing what data your account can access on a D2S instance. Once finished exploring your data, you can revoke your authorization session by logging out.

In [ ]:
# Removes access token from future requests
workspace.logout()